**Data Agumentation**

In [ ]:
pip install keras 

In [1]:
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

In [2]:
from skimage.util import random_noise
from skimage import filters
from skimage import io
import cv2
import numpy as np
import os
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
img = image.load_img('test images/letter-1.png',target_size=(600,600))

In [ ]:
plt.imshow(img)

In [3]:
datagen = ImageDataGenerator(
    rotation_range=45,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    # vertical_flip=True,
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode= 'reflect'
)

In [ ]:
#img = image.img_to_array(img)

In [ ]:
#img.shape

In [4]:
dataset = []

In [5]:
image_dir = 'test images/'
size = 128

In [6]:
my_images = os.listdir(image_dir)
for i, image_name in enumerate(my_images):
    if (image_name.split('.')[1]=='jpg') or (image_name.split('.')[1]=='png'):
        image = io.imread(image_dir+image_name)
        image = Image.fromarray(image, 'RGB')
        image = image.resize((size,size))
        dataset.append(np.array(image))
        

C:\Python310\lib\site-packages\PIL\TiffImagePlugin.py:845: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 2. 
  warnings.warn(str(msg))


In [27]:
x = np.array(dataset)
x_train_copy = x.copy()


In [ ]:
#input_batch = img.reshape(1,600,600,3)

In [38]:
i=0
for output in datagen.flow(x_train_copy,batch_size=16,save_to_dir='augmented_img',save_prefix='aug',save_format='jpg'):
    augmented_img = output[0].astype(np.uint8)
    # **Verify and Reshape if Necessary**
    if augmented_img.shape != (128, 128, 3):
        resized_img = cv2.resize(augmented_img, (img_width, img_height))  # Reshape if needed
        augmented_img = resized_img.copy()  # Update the augmented image
      
    # compressed_img = cv2.imencode('.jpg', augmented_img, [cv2.IMWRITE_JPEG_QUALITY, 50])[1]
    # degraded_img = cv2.imdecode(compressed_img, cv2.IMREAD_COLOR)
    # output = random_noise(degraded_img, mode='gaussian', var=0.01)
    # blurred_img = filters.gaussian(output, sigma=1)
    i=i+1
    if i == 10:
        break

**Split Data into training and testing**

In [29]:
from keras.preprocessing.image import load_img, img_to_array
import os
import numpy as np
from sklearn.model_selection import train_test_split
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model

In [30]:
data_dir = 'augmented_img'

# Define image dimensions
img_width, img_height = 128, 128  # Example dimensions, adjust as needed

# Load and preprocess images
images = []
for img_name in os.listdir(data_dir):
    img = load_img(os.path.join(data_dir, img_name), target_size=(128, 128))
    img_array = img_to_array(img)
    images.append(img_array)

# Convert list of images to numpy array
images = np.array(images)

# Normalize pixel values to range [0, 1]
images = images / 255.0

In [31]:
from sklearn.model_selection import train_test_split

# Split dataset into training and test sets
x_train, x_test = train_test_split(images, test_size=0.2, random_state=42)

# Check the shape of training and test sets
print("Training set shape:", x_train.shape)
print("Test set shape:", x_test.shape)


Training set shape: (92, 128, 128, 3)
Test set shape: (23, 128, 128, 3)


**Add Autoencoder**

In [23]:
input_img = Input(shape=(128, 128, 3))

In [32]:
# Encoder
x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

In [33]:
# Decoder
x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)  # Output has 3 channels for RGB images

In [34]:
# Create autoencoder model
autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='mse')  # Using Mean Squared Error loss

In [35]:
print("Expected input shape for the autoencoder model:", autoencoder.input_shape)


Expected input shape for the autoencoder model: (None, 128, 128, 3)


In [36]:
print(autoencoder.summary())


Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 conv2d_21 (Conv2D)          (None, 128, 128, 16)      448       
                                                                 
 max_pooling2d_9 (MaxPoolin  (None, 64, 64, 16)        0         
 g2D)                                                            
                                                                 
 conv2d_22 (Conv2D)          (None, 64, 64, 8)         1160      
                                                                 
 max_pooling2d_10 (MaxPooli  (None, 32, 32, 8)         0         
 ng2D)                                                           
                                                                 
 conv2d_23 (Conv2D)          (None, 32, 32, 8)         584 

In [16]:
# Train the autoencoder
autoencoder.fit(x_train, x_train,
                epochs=5,
                batch_size=128,
                shuffle=True,
                validation_data=(x_test, x_test))

Epoch 1/5


ValueError: in user code:

    File "C:\Python310\lib\site-packages\keras\src\engine\training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "C:\Python310\lib\site-packages\keras\src\engine\training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Python310\lib\site-packages\keras\src\engine\training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "C:\Python310\lib\site-packages\keras\src\engine\training.py", line 1151, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "C:\Python310\lib\site-packages\keras\src\engine\training.py", line 1209, in compute_loss
        return self.compiled_loss(
    File "C:\Python310\lib\site-packages\keras\src\engine\compile_utils.py", line 277, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    File "C:\Python310\lib\site-packages\keras\src\losses.